In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

### Importing Datasets



*   `users`: Contains user data
* `products`: Contains product data
* `orders`: Contains order data
* `order_items`: Contains order items data
* `us_events`: Contains all US events data

In [2]:
users = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/users.csv')
products = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/products.csv')
orders = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/orders.csv')
order_items = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/order_items.csv')

Since the events dataset was too big to import as one file, it was split into 5 individual files and then concatenated.

In [3]:
US_0 = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/US_events_0.csv')
US_1 = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/US_events_1.csv')
US_2 = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/US_events_2.csv')
US_3 = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/US_events_3.csv')
US_4 = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/US_events_4.csv')
US_5 = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/main/data/US_events_5.csv')

In [4]:
events = pd.concat([US_0, US_1, US_2, US_3, US_4, US_5], ignore_index=True)

In [5]:
print(f"Users:{users.shape}")
print(f"Products:{products.shape}")
print(f"Orders:{orders.shape}")
print(f"Order_Items:{order_items.shape}")
print(f"Events:{events.shape}")

Users:(100000, 15)
Products:(29120, 9)
Orders:(125226, 9)
Order_Items:(181759, 11)
Events:(541963, 13)


### Data Understanding

#### Orders
* Notice that a lot of the data for `returned_at`, `shipped_at`, `delivered_at` is `NaN` (only 35% of data is completely non-null)
* Upon changing all time columns to datetime columns and removing entries with all null data, we are left with 122163 entries

In [6]:
orders.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2


In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125226 entries, 0 to 125225
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      125226 non-null  int64 
 1   user_id       125226 non-null  int64 
 2   status        125226 non-null  object
 3   gender        125226 non-null  object
 4   created_at    125226 non-null  object
 5   returned_at   12530 non-null   object
 6   shipped_at    81461 non-null   object
 7   delivered_at  43884 non-null   object
 8   num_of_item   125226 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 8.6+ MB


In [8]:
#Changing all the "-at" columns to datetime columns
dateTime_cols = ['created_at', 'returned_at', 'delivered_at', 'shipped_at']
for column in dateTime_cols:
    orders[column] = pd.to_datetime(orders[column], errors = 'coerce')
orders = orders.dropna(subset = dateTime_cols, how = 'all')
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122163 entries, 0 to 125225
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   order_id      122163 non-null  int64              
 1   user_id       122163 non-null  int64              
 2   status        122163 non-null  object             
 3   gender        122163 non-null  object             
 4   created_at    122163 non-null  datetime64[ns, UTC]
 5   returned_at   12225 non-null   datetime64[ns, UTC]
 6   shipped_at    79454 non-null   datetime64[ns, UTC]
 7   delivered_at  42814 non-null   datetime64[ns, UTC]
 8   num_of_item   122163 non-null  int64              
dtypes: datetime64[ns, UTC](4), int64(3), object(2)
memory usage: 9.3+ MB


#### Order Items
* Notice that a lot of the data for `returned_at`, `shipped_at`, `delivered_at` is `NaN` (only 35% of data is completely non-null)
* Upon changing all time columns to datetime columns and removing entries with all null data, we are left with 177354 entries

In [9]:
order_items.head()

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,0.02


In [10]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 181759 non-null  int64  
 1   order_id           181759 non-null  int64  
 2   user_id            181759 non-null  int64  
 3   product_id         181759 non-null  int64  
 4   inventory_item_id  181759 non-null  int64  
 5   status             181759 non-null  object 
 6   created_at         181759 non-null  object 
 7   shipped_at         118281 non-null  object 
 8   delivered_at       63841 non-null   object 
 9   returned_at        18232 non-null   object 
 10  sale_price         181759 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 15.3+ MB


In [11]:
#Changing all the "-at" columns to datetime columns
dateTime_cols = ['created_at', 'returned_at', 'delivered_at', 'shipped_at']
for column in dateTime_cols:
    order_items[column] = pd.to_datetime(order_items[column], errors = 'coerce')
order_items = order_items.dropna(subset = dateTime_cols, how = 'all')
order_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177354 entries, 0 to 181758
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   id                 177354 non-null  int64              
 1   order_id           177354 non-null  int64              
 2   user_id            177354 non-null  int64              
 3   product_id         177354 non-null  int64              
 4   inventory_item_id  177354 non-null  int64              
 5   status             177354 non-null  object             
 6   created_at         177354 non-null  datetime64[ns, UTC]
 7   shipped_at         115410 non-null  datetime64[ns, UTC]
 8   delivered_at       62286 non-null   datetime64[ns, UTC]
 9   returned_at        17784 non-null   datetime64[ns, UTC]
 10  sale_price         177354 non-null  float64            
dtypes: datetime64[ns, UTC](4), float64(1), int64(5), object(1)
memory usage: 16.2+ MB


#### Products

*

In [12]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


In [13]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29120 entries, 0 to 29119
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      29120 non-null  int64  
 1   cost                    29120 non-null  float64
 2   category                29120 non-null  object 
 3   name                    29118 non-null  object 
 4   brand                   29096 non-null  object 
 5   retail_price            29120 non-null  float64
 6   department              29120 non-null  object 
 7   sku                     29120 non-null  object 
 8   distribution_center_id  29120 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


### Joining `orders` and `order_items`

In [14]:
orders_order_items = pd.merge(order_items, orders, left_on=['order_id', 'user_id'], right_on=['order_id', 'user_id'], how='left', suffixes=('_item', '_order'))
orders_order_items.head()

,id,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,shipped_at_item,delivered_at_item,returned_at_item,sale_price,status_order,gender,created_at_order,returned_at_order,shipped_at_order,delivered_at_order,num_of_item
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaT,NaT,NaT,0.02,Cancelled,F,2023-05-07 07:33:00+00:00,NaT,NaT,NaT,1
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaT,0.02,Complete,F,2023-03-14 03:59:00+00:00,NaT,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,1
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaT,0.02,Complete,F,2023-12-03 13:23:00+00:00,NaT,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,4
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaT,NaT,0.02,Shipped,F,2023-12-23 00:17:00+00:00,NaT,2023-12-24 16:44:00+00:00,NaT,4
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaT,NaT,0.02,Shipped,F,2022-06-17 17:21:00+00:00,NaT,2022-06-19 19:29:00+00:00,NaT,3


### Joining `orders_order_items` and `products`

In [15]:
orders_products = pd.merge(orders_order_items, products, left_on = ['product_id'], right_on=['id'], how='left', suffixes=('_orderproducts', '_products'))
orders_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177354 entries, 0 to 177353
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   id_orderproducts        177354 non-null  int64              
 1   order_id                177354 non-null  int64              
 2   user_id                 177354 non-null  int64              
 3   product_id              177354 non-null  int64              
 4   inventory_item_id       177354 non-null  int64              
 5   status_item             177354 non-null  object             
 6   created_at_item         177354 non-null  datetime64[ns, UTC]
 7   shipped_at_item         115410 non-null  datetime64[ns, UTC]
 8   delivered_at_item       62286 non-null   datetime64[ns, UTC]
 9   returned_at_item        17784 non-null   datetime64[ns, UTC]
 10  sale_price              177354 non-null  float64            
 11  status_order            17

### Merging all order data

In [21]:
all_orders = pd.merge(orders_products, order_items, left_on = ['inventory_item_id'], right_on = ['id'], how='left', suffixes=('', '_items'))

In [22]:
all_orders

,id_orderproducts,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,shipped_at_item,delivered_at_item,returned_at_item,...,order_id_items,user_id_items,product_id_items,inventory_item_id_items,status,created_at,shipped_at,delivered_at,returned_at,sale_price_items
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaT,...,76203.0,60802.0,678.0,298421.0,Cancelled,2023-04-29 07:06:36+00:00,NaT,NaT,NaT,16.950001
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaT,...,95158.0,75922.0,20152.0,373119.0,Complete,2022-05-22 12:10:29+00:00,2022-05-22 01:41:00+00:00,2022-05-25 18:26:00+00:00,NaT,32.990002
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaT,NaT,...,68193.0,54488.0,22522.0,267317.0,Cancelled,2024-01-14 13:33:52+00:00,NaT,NaT,NaT,16.080000
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177349,9674,6679,5325,24447,26095,Returned,2020-12-23 00:05:01+00:00,2020-12-25 22:54:00+00:00,2020-12-28 07:08:00+00:00,2020-12-30 22:43:00+00:00,...,18050.0,14380.0,15958.0,70386.0,Processing,2024-01-06 14:19:31+00:00,NaT,NaT,NaT,7.020000
177350,7801,5416,4283,24447,21078,Shipped,2022-01-12 23:51:07+00:00,2022-01-10 03:42:00+00:00,NaT,NaT,...,14606.0,11680.0,22567.0,56883.0,Complete,2022-05-11 00:05:31+00:00,2022-05-09 11:28:00+00:00,2022-05-13 23:50:00+00:00,NaT,58.000000
177351,62986,43364,34691,23546,169937,Shipped,2023-02-26 00:38:43+00:00,2023-02-25 06:53:00+00:00,NaT,NaT,...,117043.0,93509.0,18864.0,458793.0,Returned,2022-12-12 22:53:04+00:00,2022-12-11 06:06:00+00:00,2022-12-13 01:34:00+00:00,2022-12-15 01:48:00+00:00,64.949997
177352,106577,73418,58623,24447,287560,Shipped,2023-11-12 02:41:02+00:00,2023-11-14 20:44:00+00:00,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN


### Merging all data

In [24]:
all_data = pd.merge(all_orders, users, left_on=['user_id'], right_on=['id'], how='left', suffixes=('_orders', '_user'))

In [25]:
all_data.head()

,id_orderproducts,order_id,user_id,product_id,inventory_item_id,status_item,created_at_item,shipped_at_item,delivered_at_item,returned_at_item,...,gender_user,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at_user
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaT,NaT,NaT,...,F,Hebei,3863 Johnson Brooks Suite 474,74199,Jinhua,China,39.388173,115.731465,Search,2021-11-27 07:33:00+00:00
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaT,...,F,Jiangsu,5374 Courtney Parkway,215007,Yulin,China,31.278607,120.642997,Organic,2022-07-15 03:59:00+00:00
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaT,...,F,Texas,7187 Michael Forge Apt. 777,75604,Longview,United States,32.512434,-94.825486,Organic,2019-06-22 13:23:00+00:00
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaT,NaT,...,F,Hyogo,6970 Cherry Extension Apt. 442,675-0137,Kakogawa City,Japan,34.718505,134.825010,Search,2022-05-17 00:17:00+00:00
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaT,NaT,...,F,England,6522 Stephens Meadows Suite 475,LA1,Lancaster,United Kingdom,54.045973,-2.794106,Search,2021-06-23 17:21:00+00:00


In [27]:
all_orders.to_csv('all_orders.csv', index=False)
all_data.to_csv('all_data.csv', index=False)